# 결정 트리

로지스틱 회귀로 와인 분류하기

In [ ]:
# 데이터 불러오기
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [ ]:
wine.head()

처음 3개의 열(alcohol, sugar, pH)은 각각 알코올 도수, 당도, pH 값을 나타냄

네 번째 열(class)은 타깃값 - 레드와인 0, 화이트 와인 1

In [ ]:
# pandas info()메서드
# 각 열의 데이터 타입과 누락된 데이터가 있는지 확인
wine.info()

누락된 값이 존재한다면 그 데이터를 버리거나 평균값으로 채운 후 사용

어떤 방식이 최선인지 미리 알기는 어려움

In [ ]:
# describe()메서드
# 평균, 표준편차, 최소, 최대, 중간값, 1사분위, 3사분위
wine.describe()

In [ ]:
# 알코올 도수와 당도, pH 값의 스케일이 다르다
# 특성 표준화 필요
# 데이터프레임->넘파이배열
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
# 훈련 세트, 테스트 세트 나누기
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [ ]:
# 훈련 세트와 테스트 세트 크기 확인
print(train_input.shape, test_input.shape)

In [ ]:
# StandardScaler를 사용해 훈련 세트 전처리
# 같은 객체 사용해 테스트 세트 전처리
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [ ]:
# 로지스틱 회귀 모델 훈련
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_scaled, train_target)
print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))

훈련 세트와 테스트 세트의 점수가 모두 낮으니 모델이 다소 과소적합됨

In [ ]:
# 로지스틱 회귀가 학습한 계수와 절편 출력
print(lr.coef_, lr.intercept_)

사실 이 모델이 왜 저런 계수 값을 학습했는지 정확히 이해하기는 어려움

아마도 알코올 도수와 당도가 높을수록 화이트 와인일 가능성이 높고 pH가 높을수록 레드 와인일 가능성이 높다
- 하지만 정확히 이 숫자가 어떤 의미인지 설명하긴 어려움
- 다항 특성을 추가한다면 설명하기가 더욱 어려워질 것

**결정트리**모델은 이유를 설명하기 쉽다
- 결정트리 모델은 스무고개와 같다
- 사이킷런의 DecisionTreeClassifier 클래스를 사용해 결정 트리 모델 훈련 가능

결정 트리 모델을 만들 때 random_state 지정하는 이유?
- 사이킷런의 결정 트리 알고리즘은 노드에서 최적의 분할을 찾기 전에 특성의 순서를 섞는다
- 약간의 무작위성이 주입되므로 값이 일정하게 나오게 random_state 지정

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(train_scaled, train_target)
print(dt.score(train_scaled, train_target))
print(dt.score(test_scaled, test_target))

In [ ]:
# 사이킷런의 plot_tree()메서드로 트리 구조 그림으로 출력
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
plt.figure(figsize=(10,7))
plot_tree(dt)
plt.show()

맨 위의 노드를 루투 노드(root node) 끝에 달린 노드를 리프 노드(leaf node)

In [ ]:
# max_depth() 매개변수로 트리 깊이 제한에서 출력 가능
# feature_names 매개변수에 특성의 이름을 전달할 수 있음
plt.figure(figsize=(10,7))
plot_tree(dt, max_depth=1, filled=True, feature_names=['alcohol', 'sugar', 'pH'])
plt.show()

- 루트 노드의 샘플수는 5197개
- 음성 클래스(레드와인)은 1258, 양성 클래스(화이트 와인)은 3939 -> value에 이 값이 나타내져 있음

결정 트리에서 예측하는 방법은 리프 노드에서 가장 많은 클래스가 예측 클래스가 된다

**불순도**
- gini는 지니 불순도를 의미
- DecisionTreeClassifier의 criterion 매개변수의 기본값이 'gini'
- criterion 매개변수의 용도는 노드에서 데이터를 분할할 기준을 정하는 것

$
지니 불순도 = 1 - (음성 클래스 비율^2 + 양성 클래스 비율^2)
$

노드에 하나의 클래스만 있다면 지니 불순도는 0 -> 이런 노드를 순수 노드라고함

결정 트리 모델은 부모 노드와 자식 노드의 불순도 차이가 가능한 크도록 트리를 성장

부모와 자식 노드 사이의 불순도 차이를 **정보 이득**이라고 함

**정보 이득** = 부모 노드의 불순도 - 자식 노드의 불순도를 샘플 개수에 비례해 모두 더한 값

**가지치기**

In [ ]:
# 가장 간단한 가지치기 - 최대 깊이를 지정하는 것
dt = DecisionTreeClassifier(max_depth=3, random_state=42)
dt.fit(train_scaled, train_target)
print(dt.score(train_scaled, train_target))
print(dt.score(test_scaled, test_target))

In [ ]:
# 그래프 플롯
plt.figure(figsize=(10,7))
plot_tree(dt, max_depth=3, filled=True, feature_names=['alcohol', 'sugar', 'pH'])
plt.show()

결정 트리는 표준화 전처리 과정이 필요 없다

In [ ]:
# 전처리 전의 훈련 세트와 테스트 세트로 결정 트리 모델 다시 훈련
dt = DecisionTreeClassifier(max_depth=3, random_state=42)
dt.fit(train_input, train_target)
print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))

In [ ]:
# 트리 그리기
plt.figure(figsize=(10,7))
plot_tree(dt, filled=True, feature_names=['alcohol', 'sugar', 'pH'])
plt.show()

In [ ]:
# 결정트리는 어떤 특성이 가장 유용한지 나타내는 특성 중요도를 계산 가능
# feature_importances_ 속성에 저장됨
print(dt.feature_importances_)

결정 트리의 특성 중요도를 특성 선택에 활용 가능